In [1]:
#Importing the packages
import pandas as pd
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import nltk
import re,string
nltk.download('stopwords')
from nltk.corpus import stopwords
stopword=set(stopwords.words('english'))
stemmer = nltk.SnowballStemmer("english")

#read the dataset with name "Fake_Real_Data.csv" and store it in a variable df
df = pd.read_csv("C:\\Users\\Admin\\Downloads\\twitter_data.csv")

#print the shape of dataframe
print(df.shape)

#print top 5 rows
df.head(5)

C:\Users\Admin\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(24783, 7)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [2]:
nlp = spacy.load("en_core_web_lg")

In [3]:
df["labels"] = df['class'].map({0: "Hate Speech", 1: "Offensive Speech", 2: "No Hate and Offensive Speech"})
df = df[["tweet", "labels"]]

In [4]:
 def preprocess(text):
        test_list = text.split()
        text = ""
        for word in test_list:
            if word.endswith("ing"):
                word=word[:-3]
            text+=word
            text+=" "
        doc = nlp(text)
        filtered_tokens = []
        
        for token in doc:
            if token.is_stop or token.is_punct:
                continue
            filtered_tokens.append(token.lemma_)
            
        return " ".join(filtered_tokens)

In [5]:
df["tweet_new"] = df.tweet.apply(preprocess)

In [6]:
df.head()

,tweet,labels,tweet_new
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive Speech,RT @mayasolovely woman complain clean house am...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Speech,RT @mleew17 boy dats cold tyga dwn bad cuffin ...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Speech,RT @urkindofbrand Dawg RT @80sbaby4life fuck b...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Speech,RT @C_G_Anderson @viva_based look like tranny
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Speech,RT @ShenikaRoberts shit hear true faker bitch ...


In [7]:
def clean(text):
    text = str (text). lower()
    text = re. sub('[.?]', '', text)
    text = re. sub('https?://\S+|www.\S+', '', text)
    text = re. sub('<.?>+', '', text)
    text = re. sub('[%s]' % re. escape(string. punctuation), '', text)
    text = re. sub('\n', '', text)
    text = re. sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ". join(text)
    text = [stemmer. stem(word) for word in text. split(' ')]
    text=" ". join(text)
    return text

In [8]:
df["tweet_new"] = df.tweet_new.apply(clean)

In [9]:
# Load pre-trained GloVe word vectors
glove_vectors = {}  # Create a dictionary to store GloVe word vectors

# Load GloVe word vectors from the downloaded file
glove_file = "C:\\Users\\Admin\\glove.twitter.27B.200d.txt"  # Provide the path to your GloVe file
with open(glove_file, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype="float32")
        glove_vectors[word] = vector

In [10]:
# Define a function to convert a sentence to an average GloVe word vector using spaCy and GloVe
def sentence_to_vector(sentence, model, word_vectors):
    doc = model(sentence)
    vector = np.zeros(word_vectors["the"].shape)  # Initialize with a vector of zeros
    num_words = 0
    for word in doc:
        if word.text in word_vectors:
            vector += word_vectors[word.text]
            num_words += 1
    if num_words > 0:
        vector /= num_words
    return vector

In [11]:
x = np. array(df["tweet_new"])
y = np. array(df["labels"])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [12]:
# Convert training data to average GloVe word vectors
X_train_glove = [sentence_to_vector(sentence, nlp, glove_vectors) for sentence in X_train]

In [13]:
from sklearn.preprocessing import LabelEncoder
# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Split data into training and testing
X_train_glv, X_test_glv, y_train_enc, y_test_enc = train_test_split(X_train_glove, y_train_encoded, test_size=0.2, random_state=42)

In [14]:
from sklearn.svm import SVC

# Define and train an SVM classifier
clf = SVC()
clf.fit(X_train_glv, y_train_enc)

#Testing the model
y_pred = clf.predict (X_test_glv)
#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test_enc,y_pred))

# Preprocess and clean the new input
new_string = "string"
new_string = preprocess(new_string)

# Convert the new input to an average GloVe word vector
new_string_vector = sentence_to_vector(new_string, nlp, glove_vectors).reshape(1, -1)

# Predict the label
predicted_label_encoded = clf.predict(new_string_vector)
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)

print("Predicted Label:", predicted_label)

0.8909966877446552
Predicted Label: ['Offensive Speech']


In [15]:
from sklearn.ensemble import RandomForestClassifier

# Define and train an SVM classifier
clf = RandomForestClassifier()
clf.fit(X_train_glv, y_train_enc)

#Testing the model
y_pred = clf.predict (X_test_glv)
#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test_enc,y_pred))

# Preprocess and clean the new input
new_string = "string"
new_string = preprocess(new_string)

# Convert the new input to an average GloVe word vector
new_string_vector = sentence_to_vector(new_string, nlp, glove_vectors).reshape(1, -1)

# Predict the label
predicted_label_encoded = clf.predict(new_string_vector)
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)

print("Predicted Label:", predicted_label)

0.8729298404095152
Predicted Label: ['Offensive Speech']


In [16]:
from sklearn.ensemble import VotingClassifier

# Initialize individual models
dt_model = DecisionTreeClassifier()
rf_model = RandomForestClassifier()
svm_model = SVC(kernel='linear', probability=True)  # Enable probability estimation for SVM

# Define the ensemble of models
ensemble_model = VotingClassifier(estimators=[
    ('decision_tree', dt_model),
    ('random_forest', rf_model),
    ('svm', svm_model)
], voting='soft')  # Use soft voting for probability averaging

# Training the ensemble model
ensemble_model.fit(X_train_glv, y_train_enc)

# Testing the ensemble model
y_pred_ensemble = ensemble_model.predict(X_test_glv)

# Accuracy Score of ensemble model
print("Ensemble Accuracy Score:", accuracy_score(y_test_enc, y_pred_ensemble))

Ensemble Accuracy Score: 0.8566696778078892


In [24]:
# Get input from the user
user_input = input("Enter the text you want to classify: ")

# Preprocess the user input
user_input_processed = preprocess(user_input)

# Transform the preprocessed input using the TF-IDF vectorizer
user_input_vectorized = sentence_to_vector(user_input_processed, nlp, glove_vectors).reshape(1, -1)

# Make prediction using the ensemble model
prediction = ensemble_model.predict(user_input_vectorized)

# Print the predicted label
print("Predicted label:", prediction)

Enter the text you want to classify: hello
Predicted label: [2]
